The metrics calculation performed in RNAfold_Benchmark.ipynb and IPknot_Benchmark.ipynb turned out to be wrong.
Here I provide proper calculation of metrics for these tools and code snippets for further benchmarking.

In Aptamers_2D.csv [RNAfold_ref] is not accuracy, it is Taminoto score (Taminoto accuracy) used in https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6975895/

Same for [RNAfold_ref] and [RNAfold_real] in RNAfold_benchmarking.csv and [IPknotv1] in IPknot_benchmarking.csv

In [22]:
import pandas as pd

In [52]:
def find_basepairs(s):
    stack = []
    result = []

    for index, char in enumerate(s):
        if char == '(':
            stack.append(index)
        elif char == ')':
            # Если нашли закрывающую скобку, сопоставляем с последней открывающей
            if stack:
                open_index = stack.pop()
                result.append((open_index, index))

    return sorted(result)

def find_knotpairs(s):
    stack = []
    #result = {}
    result = []

    for index, char in enumerate(s):
        if char == '[':
            # Запоминаем индекс открывающей скобки
            stack.append(index)
        elif char == ']':
            # Если нашли закрывающую скобку, сопоставляем с последней открывающей
            if stack:
                open_index = stack.pop()
                #result[open_index] = index
                result.append((open_index, index))

    return sorted(result)

def find_otherpairs(s):
    stack = []
    #result = {}
    result = []

    for index, char in enumerate(s):
        if char == '{':
            # Запоминаем индекс открывающей скобки
            stack.append(index)
        elif char == '}':
            # Если нашли закрывающую скобку, сопоставляем с последней открывающей
            if stack:
                open_index = stack.pop()
                #result[open_index] = index
                result.append((open_index, index))

    return sorted(result)

def find_pairs(s):
    basepairs = find_basepairs(s)
    knotpairs = find_knotpairs(s)
    otherpairs = find_otherpairs(s)
    pairs = basepairs + knotpairs + otherpairs
    return sorted(pairs)



def possible_pairs(s):

    l = len(s)
    pairs = []
    for i in range(l):
        for j in range(i+1,l):
            pair = (i, j)
            pairs.append(pair)

    return pairs


def Full_Benchmark_2D(pred, ref, v=True):
    
    #Taminoto accuracy, adequate in case of g-quadruplexes predicted as '+' but not '[' and ']'
    tcounter = 0
    for i in range(len(ref)):
        if ref[i] == pred[i]:
            tcounter += 1

        elif (ref[i] != '.') and pred[i] == '+':
            tcounter += 1
    taminoto = tcounter/len(ref)


    #Binary classification metrics for all pairs
    pred_pairs = find_pairs(pred)
    ref_pairs = find_pairs(ref)
    poss_pairs = possible_pairs(ref)
    
    ref_notpairs = [x for x in poss_pairs if x not in ref_pairs] 
    pred_notpairs = [x for x in poss_pairs if x not in pred_pairs]
    

    TP = len(set(pred_pairs).intersection(ref_pairs))
    FP = len(pred_pairs) -TP
    TN = len(set(pred_notpairs).intersection(ref_notpairs)) 
    FN = len(pred_notpairs)-TN

    Accuracy = (TP+TN)/(TP+TN+FP+FN)
    Precision = TP/(TP+FP)
    Recall = TP/(TP+FN)
    F1_score = (2*Precision*Recall)/(Precision+Recall)

    
    if v:
        print('Taminoto accuracy = ', round(taminoto, 3))
        print('Accuracy =', round(Accuracy, 3))
        print('Precision =', round(Precision, 3))
        print('Recall =', round(Recall, 3))
        print('F1-score =', round(F1_score, 3))

    else:
        return taminoto, Accuracy, Precision, Recall, F1_score


In [40]:
test_ref = '(((((((((((((((())))))))))))))))....'
test_pred = '((((((((((((((....))))))))))))))....'
Full_Benchmark_2D(test_pred, test_ref)

Taminoto accuracy =  0.889
Accuracy = 0.997
Precision = 1.0
Recall = 0.875
F1-score = 0.933


In [48]:
test_ref = '(((((((((((((((())))))))))))))))....'
test_pred = '.'*(len(test_ref)-2) +'()' 
Full_Benchmark_2D(test_pred, test_ref)

Taminoto accuracy =  0.056
Accuracy = 0.973
Precision = 0.0
Recall = 0.0
F1-score = 0.0


On absolutely wrong prediction accuracy is very high because the number of possible pairs increases too fast (as n^n) therefore TN is much bigger than TP, FN and FP. So F1-score is much more reasonable metric.

In [53]:
#RNAfold, a bit long - 2-3 minutes 
rnafold_db = pd.read_csv('RNAfold_benchmarking.csv', index_col=0)
rnafold_pred = ''.join(list(rnafold_db['RNAfold_2D']))
ref2d = ''.join(list(rnafold_db['2D']))
Full_Benchmark_2D(rnafold_pred, ref2d)

Taminoto accuracy =  0.76
Accuracy = 1.0
Precision = 0.809
Recall = 0.558
F1-score = 0.661


In [54]:
def find_basepairs(s):
    stack = []
    result = []

    for index, char in enumerate(s):
        if char == '(':
            stack.append(index)
        elif char == ')':
            # Если нашли закрывающую скобку, сопоставляем с последней открывающей
            if stack:
                open_index = stack.pop()
                result.append((open_index, index))

    return sorted(result)

def find_knotpairs(s):
    stack = []
    #result = {}
    result = []

    for index, char in enumerate(s):
        if char == '[':
            # Запоминаем индекс открывающей скобки
            stack.append(index)
        elif char == ']':
            # Если нашли закрывающую скобку, сопоставляем с последней открывающей
            if stack:
                open_index = stack.pop()
                #result[open_index] = index
                result.append((open_index, index))

    return sorted(result)

def find_otherpairs(s):
    stack = []
    #result = {}
    result = []

    for index, char in enumerate(s):
        if char == '{':
            # Запоминаем индекс открывающей скобки
            stack.append(index)
        elif char == '}':
            # Если нашли закрывающую скобку, сопоставляем с последней открывающей
            if stack:
                open_index = stack.pop()
                #result[open_index] = index
                result.append((open_index, index))

    return sorted(result)

def find_pairs(s):
    basepairs = find_basepairs(s)
    knotpairs = find_knotpairs(s)
    otherpairs = find_otherpairs(s)
    pairs = basepairs + knotpairs + otherpairs
    return sorted(pairs)



# def possible_pairs(s):

#     l = len(s)
#     pairs = []
#     for i in range(l):
#         for j in range(i+1,l):
#             pair = (i, j)
#             pairs.append(pair)

#     return pairs

#faster benchmark which do not calculate accuracy and TN
def Full_Benchmark_2Dv2(pred, ref, v=True):
    
    #Taminoto accuracy, adequate in case of g-quadruplexes predicted as '+' but not '[' and ']'
    tcounter = 0
    for i in range(len(ref)):
        if ref[i] == pred[i]:
            tcounter += 1

        elif (ref[i] != '.') and pred[i] == '+':
            tcounter += 1
    taminoto = tcounter/len(ref)


    #Binary classification metrics for all pairs
    pred_pairs = find_pairs(pred)
    ref_pairs = find_pairs(ref)
    # poss_pairs = possible_pairs(ref)
    
    # ref_notpairs = [x for x in poss_pairs if x not in ref_pairs] 
    # pred_notpairs = [x for x in poss_pairs if x not in pred_pairs]
    

    TP = len(set(pred_pairs).intersection(ref_pairs))
    FP = len(pred_pairs)-TP
    # TN = len(set(pred_notpairs).intersection(ref_notpairs)) 
    # FN = len(pred_notpairs)-TN
    FN = len(ref_pairs) -TP

    if TP == 0:
        TP = 0.0001

    if FP == 0:
        FP = 0.0001

    # if TN == 0:
    #     TN = 0.0001

    if FN == 0:
        FN = 0.0001
    
    # print(TP, TN, FP, FN, TP+FP+TN+FN)
    # print(len(pred_pairs), len(pred_notpairs), len(ref_pairs), len(ref_notpairs), len(poss_pairs))

    # Accuracy = (TP+TN)/(TP+TN+FP+FN)
    Precision = TP/(TP+FP)
    Recall = TP/(TP+FN)
    F1_score = (2*Precision*Recall)/(Precision+Recall)

    
    if v:
        print('Taminoto accuracy = ', round(taminoto, 3))
        # print('Accuracy =', round(Accuracy, 3))
        print('Precision =', round(Precision, 3))
        print('Recall =', round(Recall, 3))
        print('F1-score =', round(F1_score, 3))

    else:
        return taminoto, Precision, Recall, F1_score


In [57]:
rnafold_db = pd.read_csv('RNAfold_benchmarking.csv', index_col=0)
rnafold_pred = ''.join(list(rnafold_db['RNAfold_2D']))
ref2d = ''.join(list(rnafold_db['2D']))
print('Overall Metrics for RNAfold predictions')
Full_Benchmark_2Dv2(rnafold_pred, ref2d)

Overall Metrics for RNAfold predictions
Taminoto accuracy =  0.76
Precision = 0.809
Recall = 0.558
F1-score = 0.661


In [68]:
#Taminoto accuracy, adequate in case of g-quadruplexes predicted as '+' but not '[' and ']'
def taminoto(pred, ref):
    tcounter = 0
    for i in range(len(ref)):
        if ref[i] == pred[i]:
            tcounter += 1

        elif (ref[i] != '.') and pred[i] == '+':
            tcounter += 1  

    return tcounter/len(ref)


def taminoto_accurate_percent(preds, refs, cutoff=0.85, v=True):
    
    acc_list = []
    acc = 0
    acc_counter = 0
    for i in range(len(preds)):
        acc = taminoto(preds[i], refs[i])
        if acc >= cutoff:
            acc_counter += 1
        acc_list.append(acc)

    if v:
        print(f'% of Accurate structures (taminoto acc. >= {cutoff}) =', round(100*acc_counter/len(preds), 2))

    else:
        return acc_counter/len(preds), acc_list

In [69]:
print('For RNAfold')
taminoto_accurate_percent(list(rnafold_db['RNAfold_2D']), list(rnafold_db['2D']))

For RNAfold
% of Accurate structures (taminoto acc. >= 0.85) = 46.38


In [71]:
print('RNAfold on G-quadruplexes')
taminoto_accurate_percent(list(rnafold_db['RNAfold_2D'])[45:], list(rnafold_db['2D'])[45:])

rnafold_db = pd.read_csv('RNAfold_benchmarking.csv', index_col=0)
rnafold_pred = ''.join(list(rnafold_db['RNAfold_2D'])[45:])
ref2d = ''.join(list(rnafold_db['2D'])[45:])
print('Overall Metrics for RNAfold predictions')
Full_Benchmark_2Dv2(rnafold_pred, ref2d)

RNAfold on G-quadruplexes
% of Accurate structures (taminoto acc. >= 0.85) = 33.33
Overall Metrics for RNAfold predictions
Taminoto accuracy =  0.697
Precision = 0.459
Recall = 0.125
F1-score = 0.196


In [73]:
ipknot_db = pd.read_csv('IPknot_benchmarking.csv', index_col=0)
ipknot_pred = ''.join(list(ipknot_db['IPknotv1_2D']))
ref2d = ''.join(list(ipknot_db['2D']))
print('Overall Metrics for IPknot predictions')
Full_Benchmark_2Dv2(ipknot_pred, ref2d)

Overall Metrics for IPknot predictions
Taminoto accuracy =  0.656
Precision = 0.79
Recall = 0.555
F1-score = 0.652


In [75]:
print('IPknot on pseudoknots')

def taminoto_pseudoknots(pred, ref):
    l = 0 
    counter = 0
    for i in range(len(ref)):
        if ((ref[i] in ('[', ']', '{', '}')) or (pred[i] in ('[', ']', '{', '}'))):
            l+=1
            if (ref[i] == pred[i]):
                counter += 1

    return counter/l

print('Taminoto accuracy =', taminoto_pseudoknots(ipknot_pred, ref2d))

IPknot on pseudoknots
Taminoto accuracy = 0.0


In [78]:
#example showing we are not mistaken 
taminoto_pseudoknots('((([)))]', '...[[].]')

0.5

In all cases IPknot v1 (level=3, model=NUPACK) failed to predict pseudokots, although prediction of classical secondary structure elements were compatible with RNAfold results. 

There are many of non-canonical base pairs in pseudoknots, thats why I consider that choosing another settings will not improve IPknot performance
